In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp

A07.1 - Correlation MSTs

Task: Build MSTs from correlations and analyze them

+ from the correlation matrices, build the Minimum Spanning Trees (see lecture)

+ assign the correct ticker (’AAPL’, ’AMZN’, ...) as an attribute to the nodes and calculate the 

+ compare the MST degree distributions of the 1 hour, 1 minute, Normal and One-Factor correlations

In [11]:
# Load data
cormats = []

cormats.append(np.load('./data/cormat_1h.npy'))
cormats.append(np.load('./data/cormat_1m.npy'))
cormats.append(np.load('./data/cormat_gaussian_1h.npy'))
cormats.append(np.load('./data/cormat_gaussian_1m.npy'))
cormats.append(np.load('./data/cormat_onefactor_1h.npy'))
cormats.append(np.load('./data/cormat_onefactor_1m.npy'))

stocknames = {}
doc = open('./data/stocknames.txt', 'r').read().splitlines()

for i in range(len(doc)):
    stocknames[i] = doc[i]

#print(stocknames)

In [5]:
eigvals = []
eigvac = []

for cormat in cormats:
    eigval, eigvec = np.linalg.eig(cormat)
    eigvals.append(eigval)
    eigvac.append(eigvac)

{0: 'AAPL', 1: 'ABBV', 2: 'ABT', 3: 'ACN', 4: 'AIG', 5: 'ALL', 6: 'AMGN', 7: 'AMZN', 8: 'APA', 9: 'APC', 10: 'AXP', 11: 'BA', 12: 'BAC', 13: 'BAX', 14: 'BIIB', 15: 'BK', 16: 'BMY', 17: 'C', 18: 'CAT', 19: 'CL', 20: 'CMCSA', 21: 'COF', 22: 'COP', 23: 'COST', 24: 'CSCO', 25: 'CVS', 26: 'CVX', 27: 'DD', 28: 'DIS', 29: 'DOW', 30: 'DVN', 31: 'EBAY', 32: 'EMC', 33: 'EMR', 34: 'EXC', 35: 'F', 36: 'FB', 37: 'FCX', 38: 'FDX', 39: 'FOXA', 40: 'GD', 41: 'GE', 42: 'GILD', 43: 'GM', 44: 'GOOG', 45: 'GS', 46: 'HAL', 47: 'HD', 48: 'HON', 49: 'HPQ', 50: 'IBM', 51: 'INTC', 52: 'JNJ', 53: 'JPM', 54: 'KO', 55: 'LLY', 56: 'LMT', 57: 'LOW', 58: 'MA', 59: 'MCD', 60: 'MDLZ', 61: 'MDT', 62: 'MET', 63: 'MMM', 64: 'MO', 65: 'MON', 66: 'MRK', 67: 'MS', 68: 'MSFT', 69: 'NKE', 70: 'NOV', 71: 'NSC', 72: 'ORCL', 73: 'OXY', 74: 'PEP', 75: 'PFE', 76: 'PG', 77: 'PM', 78: 'QCOM', 79: 'RTN', 80: 'SBUX', 81: 'SLB', 82: 'SO', 83: 'SPG', 84: 'T', 85: 'TGT', 86: 'TWX', 87: 'TXN', 88: 'UNH', 89: 'UNP', 90: 'UPS', 91: 'USB', 9

A07.2 - Conclusions
Task: finish the analysis by drawing conclusions Answer the following questions (for both timescales):

1. Is there a ‘market mode’?

2. Which are the 5 stocks with highest degree on the 1 minute and 1 hour MST?

3. Is the degree distribution of MSTs qualitatively the same at 1 minute and 1 hour?

4. Is the degree distribution of MSTs well reproduced by the null Gaussian and One-Factor models?